# Correlation

In this notebook, we compute the correlations between the time-series of the administrative regions of a selected country. Through this analysis, we can identify time-series with high correlations between them and then decide to exclude some for subsequent analyzes.

In [1]:
import dataframe_image as dfi
import pandas as pd
import numpy as np

In [2]:
COUNTRY = "Burkina Faso"

In [3]:
PATH_TO_DATA_FOLDER = f"../Dataset time-series/output_data/{COUNTRY}/"

## Time-series dataset

In [4]:
# Load the time-series data (daily interpolation).
df = pd.read_csv(PATH_TO_DATA_FOLDER + COUNTRY + "-day.csv", header = [0, 1], index_col = 0)
df.index.name = "Datetime"
df.index = pd.to_datetime(df.index)
freq = "D"
df.index.freq = freq

In [5]:
df.columns.get_level_values(1).unique()

Index(['1 Month Anomaly Rainfalls (%)', '3 Months Anomaly Rainfalls (%)',
       'Code', 'Exchange rate', 'FCG', 'Fatalities', 'Lat', 'Lon', 'NDVI',
       'NDVI Anomaly', 'Population', 'Price cereals and tubers',
       'Rainfalls (mm)', 'Ramadan', 'rCSI'],
      dtype='object', name='Indicator')

In [6]:
# Consider only some indicators for the analysis with Spearman correlation.
INDICATORS_TO_CONSIDER = ["FCG", "1 Month Anomaly Rainfalls (%)", "3 Months Anomaly Rainfalls (%)", 
                          "NDVI", "Price cereals and tubers", "Fatalities", "NDVI Anomaly", 
                          "rCSI", "Rainfalls (mm)"] # "Exchange rate"

df = df.loc[:, df.columns.get_level_values(1).isin(INDICATORS_TO_CONSIDER)]
df.head()

AdminStrata             Boucle-Du-Mouhoun                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-09-08                     104.747834                     106.271378   
2019-09-09                     104.800365                     106.240116   
2019-09-10                     104.881777                     106.220459   
2019-09-11                     104.988996                     106.210471   
2019-09-12                     105.122024                     106.210151   

AdminStrata                                             \
Indicator        FCG Fatalities      NDVI NDVI Anomaly   
Datetime                                                 
2019-09-08   30.9217        4.0  0.496905    95.011603   
2019-09-09   31.1039        4.0  0.500725    95.044700   
2019-09-10   32.0303        4.0  0.504537    95.083381   
2019-09-11   32.1673        4.0  0.508317    95.128519   
2019-09-12   31.7701        4.0  0.512066    95.180114   

AdminStrata                                                   \
Indicator   Price cereals and tubers Rainfalls (mm)     rCSI   
Datetime                                                       
2019-09-08                  0.077833       424.5106  19.4995   
2019-09-09                  0.078059       426.2755  19.5674   
2019-09-10                  0.078284       428.0404  20.4007   
2019-09-11                  0.078509       428.2382  20.4056   
2019-09-12                  0.078734       428.4360  19.2385   

AdminStrata                      Cascades  ...    Sahel  \
Indicator   1 Month Anomaly Rainfalls (%)  ...     rCSI   
Datetime                                   ...            
2019-09-08                     120.695958  ...  23.6735   
2019-09-09                     120.653097  ...  23.8560   
2019-09-10                     120.554396  ...  24.6310   
2019-09-11                     120.442910  ...  23.3179   
2019-09-12                     120.318638  ...  23.1220   

AdminStrata                     Sud-Ouest                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-09-08                     117.742597                     117.518080   
2019-09-09                     117.536886                     117.497259   
2019-09-10                     117.317641                     117.473018   
2019-09-11                     117.088827                     117.445525   
2019-09-12                     116.850446                     117.414779   

AdminStrata                                             \
Indicator        FCG Fatalities      NDVI NDVI Anomaly   
Datetime                                                 
2019-09-08   36.2381        0.0  0.659100   100.496780   
2019-09-09   35.8286        0.0  0.660844   100.491038   
2019-09-10   36.2542        0.0  0.662560   100.485497   
2019-09-11   35.8838        0.0  0.664241   100.480693   
2019-09-12   34.9950        0.0  0.665889   100.476627   

AdminStrata                                                   
Indicator   Price cereals and tubers Rainfalls (mm)     rCSI  
Datetime                                                      
2019-09-08                  0.295372       465.6788  17.7619  
2019-09-09                  0.294013       467.8467  17.5612  
2019-09-10                  0.292654       470.0146  17.8142  
2019-09-11                  0.291296       471.5916  17.3366  
2019-09-12                  0.289937       473.1686  17.2483  

[5 rows x 117 columns]

In [7]:
df.rename({"1 Month Anomaly Rainfalls (%)": "omonthrainfalls", 
           "3 Months Anomaly Rainfalls (%)": "tmonthrainfalls", 
           "Exchange rate": "exchange", 
           "FCG": "fcg",
           "Fatalities": "fatalities",
           "NDVI Anomaly": "ndvianonaly",
           "NDVI": "ndvi",
           "Price cereals and tubers": "pricecereals", 
           "Rainfalls (mm)": "rainfalls", 
           "rCSI": "rcsi"}, axis = 1, level = 1, inplace = True)

## Correlations

### Nature indicators

In [8]:
select = df.columns.get_level_values(1).isin(["ndvi", "ndvianonaly", "rainfalls", "omonthrainfalls", "tmonthrainfalls"])
df_nature = df.loc[:, select]
df_nature.head()

AdminStrata Boucle-Du-Mouhoun                                                  \
Indicator     omonthrainfalls tmonthrainfalls      ndvi ndvianonaly rainfalls   
Datetime                                                                        
2019-09-08         104.747834      106.271378  0.496905   95.011603  424.5106   
2019-09-09         104.800365      106.240116  0.500725   95.044700  426.2755   
2019-09-10         104.881777      106.220459  0.504537   95.083381  428.0404   
2019-09-11         104.988996      106.210471  0.508317   95.128519  428.2382   
2019-09-12         105.122024      106.210151  0.512066   95.180114  428.4360   

AdminStrata        Cascades                                                  \
Indicator   omonthrainfalls tmonthrainfalls      ndvi ndvianonaly rainfalls   
Datetime                                                                      
2019-09-08       120.695958      114.756580  0.639206   97.573089  515.9717   
2019-09-09       120.653097      114.893976  0.640395   97.538516  518.5116   
2019-09-10       120.554396      115.026871  0.641564   97.504471  521.0515   
2019-09-11       120.442910      115.151260  0.642714   97.471181  521.1361   
2019-09-12       120.318638      115.267143  0.643843   97.438646  521.2207   

AdminStrata  ...           Sahel                                        \
Indicator    ... omonthrainfalls tmonthrainfalls      ndvi ndvianonaly   
Datetime     ...                                                         
2019-09-08   ...      107.588258      105.747809  0.285234   98.873611   
2019-09-09   ...      107.621997      105.809922  0.288970   99.152110   
2019-09-10   ...      107.640708      105.877591  0.292720   99.438098   
2019-09-11   ...      107.668223      105.952952  0.296412   99.724997   
2019-09-12   ...      107.704543      106.036006  0.300046  100.012805   

AdminStrata                 Sud-Ouest                                        \
Indicator   rainfalls omonthrainfalls tmonthrainfalls      ndvi ndvianonaly   
Datetime                                                                      
2019-09-08   284.6803      117.742597      117.518080  0.659100  100.496780   
2019-09-09   283.9940      117.536886      117.497259  0.660844  100.491038   
2019-09-10   283.3077      117.317641      117.473018  0.662560  100.485497   
2019-09-11   282.0858      117.088827      117.445525  0.664241  100.480693   
2019-09-12   280.8639      116.850446      117.414779  0.665889  100.476627   

AdminStrata            
Indicator   rainfalls  
Datetime               
2019-09-08   465.6788  
2019-09-09   467.8467  
2019-09-10   470.0146  
2019-09-11   471.5916  
2019-09-12   473.1686  

[5 rows x 65 columns]

In [9]:
# Compute the correlation between the nature time-series for the various administrative regions: we then obtain a list of correlation matrices equal to the number of administrative regions.
corr_matrices_nature = list()

def correlation_matrices_nature(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman")
    corr_matrices_nature.append(mtrx)

df_nature.groupby(level = 0, axis = 1).apply(correlation_matrices_nature);

In [10]:
# We take the mean of the various correlation matrices (administrative regions).
CORR_nature_mean = pd.DataFrame(np.mean(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
corr = CORR_nature_mean.style.background_gradient(cmap = "coolwarm")
#corr.export_png(f"./output_images/nature_indicators_{COUNTRY}.png")
corr

Indicator,omonthrainfalls,tmonthrainfalls,ndvi,ndvianonaly,rainfalls
Indicator,,,,,
omonthrainfalls,1.000000,0.391697,0.499572,0.265261,0.289781
tmonthrainfalls,0.391697,1.000000,0.300936,0.723721,-0.379321
ndvi,0.499572,0.300936,1.000000,0.211057,0.557809
ndvianonaly,0.265261,0.723721,0.211057,1.000000,-0.457135
rainfalls,0.289781,-0.379321,0.557809,-0.457135,1.000000


In [11]:
# We take the median of the various correlation matrices (administrative regions).
CORR_nature_median = pd.DataFrame(np.median(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
CORR_nature_median.style.background_gradient(cmap = "coolwarm")

Indicator,omonthrainfalls,tmonthrainfalls,ndvi,ndvianonaly,rainfalls
Indicator,,,,,
omonthrainfalls,1.000000,0.333895,0.517471,0.320620,0.272036
tmonthrainfalls,0.333895,1.000000,0.311790,0.719357,-0.388361
ndvi,0.517471,0.311790,1.000000,0.289576,0.562115
ndvianonaly,0.320620,0.719357,0.289576,1.000000,-0.576420
rainfalls,0.272036,-0.388361,0.562115,-0.576420,1.000000


In [12]:
# We take the variance of the various correlation matrices (administrative regions).
CORR_nature_variance = pd.DataFrame(np.var(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
CORR_nature_variance.style.background_gradient(cmap = "coolwarm")

Indicator,omonthrainfalls,tmonthrainfalls,ndvi,ndvianonaly,rainfalls
Indicator,,,,,
omonthrainfalls,0.000000,0.016766,0.019931,0.026266,0.007057
tmonthrainfalls,0.016766,0.000000,0.071240,0.015151,0.031186
ndvi,0.019931,0.071240,0.000000,0.191218,0.008831
ndvianonaly,0.026266,0.015151,0.191218,0.000000,0.112381
rainfalls,0.007057,0.031186,0.008831,0.112381,0.000000


### All indicators

In [13]:
# Compute the correlation between all the time-series for the various administrative regions: we then obtain a list of correlation matrices equal to the number of administrative regions.
corr_matrices = list()

def correlation_matrices(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman").values
    if not np.isnan(np.sum(mtrx)): # nan if the time-series is completly flat (fatalities).
        corr_matrices.append(mtrx)

df.groupby(level = 0, axis = 1).apply(correlation_matrices);

In [14]:
# We take the mean of the various correlation matrices (administrative regions).
CORR_mean = pd.DataFrame(np.mean(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
corr = CORR_mean.style.background_gradient(cmap = "coolwarm")
corr.export_png(f"./output_images/all_indicators_{COUNTRY}.png")
corr

Indicator,omonthrainfalls,tmonthrainfalls,fcg,fatalities,ndvi,ndvianonaly,pricecereals,rainfalls,rcsi
Indicator,,,,,,,,,
omonthrainfalls,1.000000,0.400234,-0.168116,0.041829,0.491648,0.261971,-0.144959,0.291260,0.422878
tmonthrainfalls,0.400234,1.000000,-0.092370,-0.048482,0.282125,0.724085,-0.269807,-0.380460,0.383398
fcg,-0.168116,-0.092370,1.000000,0.010877,-0.604525,-0.024827,0.023751,-0.345928,-0.357319
fatalities,0.041829,-0.048482,0.010877,1.000000,-0.125471,-0.079703,-0.202539,-0.044599,-0.148381
ndvi,0.491648,0.282125,-0.604525,-0.125471,1.000000,0.184311,0.249726,0.567856,0.647162
ndvianonaly,0.261971,0.724085,-0.024827,-0.079703,0.184311,1.000000,-0.231702,-0.460831,0.335320
pricecereals,-0.144959,-0.269807,0.023751,-0.202539,0.249726,-0.231702,1.000000,0.478164,0.052859
rainfalls,0.291260,-0.380460,-0.345928,-0.044599,0.567856,-0.460831,0.478164,1.000000,0.240257
rcsi,0.422878,0.383398,-0.357319,-0.148381,0.647162,0.335320,0.052859,0.240257,1.000000


In [15]:
# We take the median of the various correlation matrices (administrative regions).
CORR_median = pd.DataFrame(np.median(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
CORR_median.style.background_gradient(cmap = "coolwarm")

Indicator,omonthrainfalls,tmonthrainfalls,fcg,fatalities,ndvi,ndvianonaly,pricecereals,rainfalls,rcsi
Indicator,,,,,,,,,
omonthrainfalls,1.000000,0.336190,-0.215575,0.022977,0.491258,0.328554,-0.104594,0.288773,0.434830
tmonthrainfalls,0.336190,1.000000,-0.092257,0.017010,0.303799,0.751918,-0.269775,-0.400019,0.424601
fcg,-0.215575,-0.092257,1.000000,-0.026069,-0.642850,-0.111064,0.064011,-0.377595,-0.368609
fatalities,0.022977,0.017010,-0.026069,1.000000,-0.189059,-0.017419,-0.289786,-0.101159,-0.216671
ndvi,0.491258,0.303799,-0.642850,-0.189059,1.000000,0.262325,0.238073,0.568179,0.673952
ndvianonaly,0.328554,0.751918,-0.111064,-0.017419,0.262325,1.000000,-0.308422,-0.581967,0.388747
pricecereals,-0.104594,-0.269775,0.064011,-0.289786,0.238073,-0.308422,1.000000,0.505593,0.050976
rainfalls,0.288773,-0.400019,-0.377595,-0.101159,0.568179,-0.581967,0.505593,1.000000,0.271830
rcsi,0.434830,0.424601,-0.368609,-0.216671,0.673952,0.388747,0.050976,0.271830,1.000000


In [16]:
# We take the variance of the various correlation matrices (administrative regions).
CORR_variance = pd.DataFrame(np.var(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
CORR_variance.style.background_gradient(cmap = "coolwarm")

Indicator,omonthrainfalls,tmonthrainfalls,fcg,fatalities,ndvi,ndvianonaly,pricecereals,rainfalls,rcsi
Indicator,,,,,,,,,
omonthrainfalls,0.000000,0.017216,0.046816,0.104061,0.020776,0.028314,0.062858,0.007617,0.012637
tmonthrainfalls,0.017216,0.000000,0.091058,0.086667,0.072576,0.016412,0.078317,0.033768,0.058903
fcg,0.046816,0.091058,0.000000,0.119963,0.022207,0.168423,0.070358,0.074707,0.089288
fatalities,0.104061,0.086667,0.119963,0.000000,0.173220,0.091626,0.125506,0.126856,0.109364
ndvi,0.020776,0.072576,0.022207,0.173220,0.000000,0.197853,0.014554,0.008255,0.036738
ndvianonaly,0.028314,0.016412,0.168423,0.091626,0.197853,0.000000,0.088265,0.121569,0.138489
pricecereals,0.062858,0.078317,0.070358,0.125506,0.014554,0.088265,0.000000,0.024167,0.108708
rainfalls,0.007617,0.033768,0.074707,0.126856,0.008255,0.121569,0.024167,0.000000,0.113839
rcsi,0.012637,0.058903,0.089288,0.109364,0.036738,0.138489,0.108708,0.113839,0.000000
